In [79]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

device = torch.device('mps')
print(device)

2.1.0
mps


In [126]:
import tsl
import copy
import torch
import scipy
import numpy as np
import pandas as pd

print(f"tsl version  : {tsl.__version__}")
print(f"torch version: {torch.__version__}")

pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(edgeitems=3, precision=3)
torch.set_printoptions(edgeitems=2, precision=3)

# Utility functions ################
def print_matrix(matrix):
    return pd.DataFrame(matrix)

def print_model_size(model):
    tot = sum([p.numel() for p in model.parameters() if p.requires_grad])
    out = f"Number of model ({model.__class__.__name__}) parameters:{tot:10d}"
    print("=" * len(out))
    print(out)

tsl version  : 0.9.5
torch version: 2.1.0


In [106]:
df = pd.read_hdf('data/LCL_12month.h5')
df_numpy = copy.deepcopy(df)


In [107]:
df.head()

,MAC000002,MAC000003,MAC000005,MAC000007,MAC000009,MAC000010,MAC000011,MAC000013,MAC000014,MAC000017,...,MAC000123,MAC000124,MAC000125,MAC000127,MAC000128,MAC000129,MAC000130,MAC000131,MAC000132,MAC000133
1357000200000000000,0.24,1.92,0.11,0.20,0.18,0.45,0.30,0.35,0.03,0.04,...,0.10,0.37,0.10,0.32,0.05,0.06,0.03,0.14,0.10,0.23
1357002000000000000,0.19,1.81,0.07,0.16,0.15,0.50,0.66,0.28,0.03,0.08,...,0.11,0.26,0.13,0.21,0.04,0.05,0.03,0.16,0.10,0.36
1357003800000000000,0.23,0.80,0.04,0.21,0.09,0.62,0.68,0.14,0.02,0.04,...,0.09,0.20,0.11,0.18,0.06,0.06,0.04,0.23,0.13,0.30
1357005600000000000,0.18,1.24,0.04,0.19,0.09,0.20,0.66,0.14,0.03,0.04,...,0.05,0.14,0.13,0.20,0.04,0.07,0.04,0.25,0.08,0.20
1357007400000000000,0.23,0.92,0.03,0.13,0.14,0.18,0.77,0.10,0.03,0.07,...,0.09,0.12,0.11,0.19,0.04,0.06,0.02,0.12,0.08,0.83


In [108]:
df.index = pd.to_datetime(df.index)

#### Building graph based on DTW distance

In [109]:
df_down_sampling = df.resample('1D').sum()
df_down_sampling

,MAC000002,MAC000003,MAC000005,MAC000007,MAC000009,MAC000010,MAC000011,MAC000013,MAC000014,MAC000017,...,MAC000123,MAC000124,MAC000125,MAC000127,MAC000128,MAC000129,MAC000130,MAC000131,MAC000132,MAC000133
2013-01-01,10.58,16.77,4.24,12.92,9.75,31.82,17.06,8.05,4.23,7.97,...,7.22,9.91,4.98,10.93,5.09,2.62,3.85,15.14,10.88,30.72
2013-01-02,13.30,19.98,5.70,8.03,7.99,32.56,14.78,7.28,6.21,7.47,...,9.11,8.65,8.25,9.82,5.14,4.37,4.20,11.99,15.54,21.03
2013-01-03,10.07,23.52,5.18,9.72,11.15,30.77,19.95,5.95,5.73,7.70,...,7.92,10.36,12.68,10.89,5.49,17.72,3.41,11.00,16.17,21.27
2013-01-04,9.86,20.95,6.19,9.82,8.61,33.31,15.89,6.46,7.08,6.72,...,7.39,8.38,11.89,11.12,4.92,11.24,3.65,11.45,16.56,23.26
2013-01-05,11.17,22.06,5.37,10.97,8.15,30.31,19.87,6.13,3.74,3.61,...,7.21,12.20,17.86,8.71,5.04,10.04,3.58,7.60,12.11,22.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-12-27,9.12,31.53,6.93,10.14,9.14,46.52,16.14,6.70,7.55,7.08,...,9.13,11.47,10.00,4.35,5.18,2.53,5.90,13.56,20.86,24.11
2013-12-28,13.66,10.56,5.68,21.11,8.92,46.14,13.16,6.30,8.32,5.83,...,7.99,9.37,8.48,5.25,5.13,2.41,5.38,11.10,14.64,21.60
2013-12-29,14.98,9.75,2.98,9.16,11.79,38.42,16.54,5.71,6.53,9.06,...,8.27,7.16,8.73,11.23,9.34,2.24,4.66,14.68,14.16,28.49
2013-12-30,15.08,10.72,5.00,9.71,12.55,43.87,11.17,5.12,7.11,6.52,...,11.39,12.43,13.98,12.33,4.84,2.35,3.73,14.19,15.30,15.62


In [110]:
from dtw import dtw
import scipy.spatial.distance as sd
numpy_df = np.array(df_down_sampling).T
print(numpy_df.shape)
def dtw_distance(ts_1, ts_2):
    return dtw(ts_1, ts_2, keep_internals=True).normalizedDistance

distance_table = sd.squareform(sd.pdist(numpy_df, dtw_distance))
distance_table.shape

(100, 365)


(100, 100)

In [129]:
import scipy.special


softmax_distance = scipy.special.softmax(-distance_table, axis=1)


In [132]:
rows, cols = np.nonzero(softmax_distance)
edge_index = np.vstack((rows, cols))
edge_weights = softmax_distance[rows, cols]


(2, 10000)
(10000,)


## Building a PyTorch-ready dataset
In this section we will use [`tsl.data.SpatioTemporalDataset`](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/data_pytorch_datasets.html#tsl.data.SpatioTemporalDataset) to prepare dataset.

In particular, a `SpatioTemporalDataset` object can be used to achieve the following:
* Perform **data manipulation** operations required to feed the data to a PyTorch module (e.g., casting data to `torch.tensor`, handling possibly different `shapes`, synchronizing temporal data).
* Create **`(input, target)` samples** for supervised learning following the [**sliding window**](https://torch-spatiotemporal.readthedocs.io/en/latest/usage/spatiotemporal_dataset.html#sliding-window) approach.
* Define how data should be **arranged** in a **spatiotemporal graph signal** (e.g.,  which are the inputs and targets, how node attributes and covariates variables are mapped into a single graph).
* **Preprocess** data before creating a **spatiotemporal graph signal** by appling [**transformations**](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/transforms.html) or [**scaling**](https://torch-spatiotemporal.readthedocs.io/en/latest/modules/data_preprocessing.html) operations.


In [134]:
from tsl.data import SpatioTemporalDataset


torch_dataset = SpatioTemporalDataset(target=df_numpy,
                                      connectivity=(edge_index, edge_weights),
                                      horizon=12,
                                      window=12,
                                      stride=1)

print(torch_dataset)

SpatioTemporalDataset(n_samples=17449, n_nodes=100, n_channels=1)


In [135]:
sample = torch_dataset[0]
print(sample)


Data(
  input=(x=[t=12, n=100, f=1], edge_index=[2, e=10000], edge_weight=[e=10000]),
  target=(y=[t=12, n=100, f=1]),
  has_mask=False
)


### Input and target

In [136]:
sample.input.to_dict().keys()

dict_keys(['x', 'edge_index', 'edge_weight'])

In [137]:
if sample.has_transform:
    print(sample.transform)
else:
    print("Sample has no transformation functions.")

Sample has no transformation functions.


In [138]:
sample.pattern

{'x': 't n f', 'edge_index': '2 e', 'edge_weight': 'e', 'y': 't n f'}

### Batching sample

In [139]:
batch = torch_dataset[:5]
print(batch.pattern)
print(batch)

{'x': 'b t n f', 'edge_index': '2 e', 'edge_weight': 'e', 'y': 'b t n f'}
StaticBatch(
  input=(x=[b=5, t=12, n=100, f=1], edge_index=[2, e=10000], edge_weight=[e=10000]),
  target=(y=[b=5, t=12, n=100, f=1]),
  has_mask=False
)


### Preparing the dataset for training

In [140]:
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
from tsl.data.preprocessing import StandardScaler

# Normalize data using mean and std computed over time and node dimensions
scalers = {'target': StandardScaler(axis=(0, 1))}

# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
splitter = TemporalSplitter(val_len=0.1, test_len=0.2)

dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,
)

print(dm)

SpatioTemporalDataModule(train_len=None, val_len=None, test_len=None, scalers=[target], batch_size=64)


In [141]:
dm.setup()
print(dm)


SpatioTemporalDataModule(train_len=12552, val_len=1384, test_len=3489, scalers=[target], batch_size=64)


In [142]:
from tsl.nn.models.stgn.graph_wavenet_model import GraphWaveNetModel

gw = GraphWaveNetModel(input_size=torch_dataset.n_channels,
                       horizon=torch_dataset.horizon,
                       output_size=torch_dataset.n_channels,
                       n_nodes=torch_dataset.n_nodes)

In [151]:
import torch.nn as nn

from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange  # reshape data with Einstein notation


class TimeThenSpaceModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 hidden_size: int = 32,
                 rnn_layers: int = 1,
                 gnn_kernel: int = 2):
        super(TimeThenSpaceModel, self).__init__()

        self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)

        self.time_nn = RNN(input_size=hidden_size,
                           hidden_size=hidden_size,
                           n_layers=rnn_layers,
                           cell='lstm',
                           return_only_last_state=True)

        self.space_nn = DiffConv(in_channels=hidden_size,
                                 out_channels=hidden_size,
                                 k=gnn_kernel)

        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)
        self.first_time = True

    def forward(self, x, edge_index, edge_weight):
        # x: [batch time nodes features]
        if self.first_time:
          print("Shape of x")
          print(x.shape)
          self.first_time = False

        x_enc = self.encoder(x)  # linear encoder: x_enc = xΘ + b
        x_emb = x_enc + self.node_embeddings()  # add node-identifier embeddings
        h = self.time_nn(x_emb)  # temporal processing: x=[b t n f] -> h=[b n f]
        z = self.space_nn(h, edge_index, edge_weight)  # spatial processing
        x_out = self.decoder(z)  # linear decoder: z=[b n f] -> x_out=[b n t⋅f]
        x_horizon = self.rearrange(x_out)
        return x_horizon

We can play with hyperparameters and make an instance of our model.

In [167]:
hidden_size = 32   #@param
rnn_layers = 1     #@param
gnn_kernel = 2     #@param

input_size = torch_dataset.n_channels   # 1 channel
n_nodes = torch_dataset.n_nodes         # 207 nodes
horizon = torch_dataset.horizon         # 12 time steps

stgnn = TimeThenSpaceModel(input_size=input_size,
                           n_nodes=n_nodes,
                           horizon=horizon,
                           hidden_size=hidden_size,
                           rnn_layers=rnn_layers,
                           gnn_kernel=gnn_kernel)
print(stgnn)
print_model_size(stgnn)

TimeThenSpaceModel(
  (encoder): Linear(in_features=1, out_features=32, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=100, embedding_size=32)
  (time_nn): RNN(
    (rnn): LSTM(32, 32)
  )
  (space_nn): DiffConv(32, 32)
  (decoder): Linear(in_features=32, out_features=12, bias=True)
  (rearrange): Rearrange('b n (t f) -> b t n f', t=12)
)
Number of model (TimeThenSpaceModel) parameters:     17260


### Setting up training

#### The predictor

In [143]:
from tsl.metrics.torch import MaskedMAE, MaskedMAPE
from tsl.engines import Predictor

loss_fn = MaskedMAE()

metrics = {'mae': MaskedMAE(),
           'mape': MaskedMAPE(),
           'mae_at_15': MaskedMAE(at=2),  # '2' indicates the third time step,
                                          # which correspond to 15 minutes ahead
           'mae_at_30': MaskedMAE(at=5),
           'mae_at_60': MaskedMAE(at=11)}

# setup predictor
predictor = Predictor(
    model=gw,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=loss_fn,               # which loss function to be used
    metrics=metrics                # metrics to be logged during train/val/test
)

In [168]:
predictor_stgnn = Predictor(
    model=stgnn,
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=loss_fn,               # which loss function to be used
    metrics=metrics                # metrics to be logged during train/val/test
)

In [144]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger(save_dir="logs", name="graph_wavenet", version=0)

In [164]:
logger_stgnn = TensorBoardLogger(save_dir="logs", name="stgnn", version=0)

In [150]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 8121), started 2 days, 23:47:02 ago. (Use '!kill 8121' to kill it.)

#### Setup Lighting trainer

In [148]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    dirpath='logs',
    save_top_k=1,
    monitor='val_mae',
    mode='min',
)

# trainer = pl.Trainer(max_epochs=10,
#                      logger=logger,
#                      accelerator="mps" if torch.backends.mps.is_available() else "cpu",
#                      devices=1,
#                      limit_train_batches=10,  # end an epoch after 100 updates
#                      callbacks=[checkpoint_callback])

trainer = pl.Trainer(max_epochs=10,
                     logger=logger,
                     accelerator="cpu",
                     devices=1,
                     limit_train_batches=10,  # end an epoch after 100 updates
                     callbacks=[checkpoint_callback])



trainer.fit(predictor, datamodule=dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type              | Params
----------------------------------------------------
0 | loss_fn       | MaskedMAE         | 0     
1 | train_metrics | MetricCollection  | 0     
2 | val_metrics   | MetricCollection  | 0     
3 | test_metrics  | MetricCollection  | 0     
4 | model         | GraphWaveNetModel | 299 K 
----------------------------------------------------
299 K     Trainable params
0         Non-trainable params
299 K     Total params
1.196     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [177]:
checkpoint_callback_stgnn = ModelCheckpoint(
    dirpath='logs',
    save_top_k=1,
    monitor='val_mae',
    mode='min',
)

trainer_stgnn = pl.Trainer(max_epochs=100,
                     logger=logger,
                    #  accelerator="mps" if torch.backends.mps.is_available() else "cpu",
                     accelerator="cpu",
                     devices=1,
                     limit_train_batches=10,  # end an epoch after 100 updates
                     callbacks=[checkpoint_callback_stgnn])


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [178]:
trainer_stgnn.fit(predictor_stgnn, datamodule=dm)


  | Name          | Type               | Params
-----------------------------------------------------
0 | loss_fn       | MaskedMAE          | 0     
1 | train_metrics | MetricCollection   | 0     
2 | val_metrics   | MetricCollection   | 0     
3 | test_metrics  | MetricCollection   | 0     
4 | model         | TimeThenSpaceModel | 17.3 K
-----------------------------------------------------
17.3 K    Trainable params
0         Non-trainable params
17.3 K    Total params
0.069     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/envs/Time_Series/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


## Testing
<hr>


Now let's see how the trained model behaves on new unseen data.

In [149]:
predictor.load_model(checkpoint_callback.best_model_path)
predictor.freeze()

trainer.test(predictor, datamodule=dm)

Predictor with already instantiated model is loading a state_dict from /Users/quocviet.nguyen/Time_Series/Graph_ST/logs/epoch=7-step=80.ckpt. Cannot  check if model hyperparameters are the same.
/opt/anaconda3/envs/Time_Series/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.14331026375293732    │
│         test_mae          │    0.1433102935552597     │
│      test_mae_at_15       │    0.12891197204589844    │
│      test_mae_at_30       │    0.14780747890472412    │
│      test_mae_at_60       │    0.16278523206710815    │
│         test_mape         │    0.5629060864448547     │
└───────────────────────────┴───────────────────────────┘

[{'test_mae': 0.1433102935552597,
  'test_mae_at_15': 0.12891197204589844,
  'test_mae_at_30': 0.14780747890472412,
  'test_mae_at_60': 0.16278523206710815,
  'test_mape': 0.5629060864448547,
  'test_loss': 0.14331026375293732}]

In [179]:
predictor_stgnn.load_model(checkpoint_callback_stgnn.best_model_path)
predictor_stgnn.freeze()

trainer_stgnn.test(predictor_stgnn, datamodule=dm)

Predictor with already instantiated model is loading a state_dict from /Users/quocviet.nguyen/Time_Series/Graph_ST/logs/epoch=99-step=1000.ckpt. Cannot  check if model hyperparameters are the same.
/opt/anaconda3/envs/Time_Series/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     0.137766495347023     │
│         test_mae          │    0.13776648044586182    │
│      test_mae_at_15       │    0.12793321907520294    │
│      test_mae_at_30       │    0.14296875894069672    │
│      test_mae_at_60       │    0.15193063020706177    │
│         test_mape         │    0.5723749995231628     │
└───────────────────────────┴───────────────────────────┘

[{'test_mae': 0.13776648044586182,
  'test_mae_at_15': 0.12793321907520294,
  'test_mae_at_30': 0.14296875894069672,
  'test_mae_at_60': 0.15193063020706177,
  'test_mape': 0.5723749995231628,
  'test_loss': 0.137766495347023}]

Cool! We succeeded in creating our first simple – yet effective – Spatiotemporal GNN!

🥷 We are now **tsl ninjas**. We learned how to:

* Load benchmark datasets
* Organize data for processing
* Preprocess the data
* Build a Spatiotemporal GNN
* Train and evaluate models

We hope you enjoyed this introduction to tsl, now go and build the game-changer STGNN out there 🌎

🧡 The tsl team